In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [44]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


def execute_global_registration(
        source_down,
        target_down,
        source_fpfh,
        target_fpfh,
        voxel_size
):
    distance_threshold = voxel_size * 1.5
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source=source_down,
        target=target_down,
        source_feature=source_fpfh,
        target_feature=target_fpfh,
        mutual_filter=True,
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        # estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        ransac_n=3,
        checkers=[
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
        ],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(200000, 0.99999)
    )
    return result


def preprocess_point_cloud(pcd, voxel_size):
    # pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down = pcd
    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh


In [45]:
# ply_source = "D:/sfm_dense/street_1_0_real_time/target.ply"
ply_source = "D:/sfm_dense/street_2_0/dense/0/fused_filtered_down_aligned.ply"

# ply_target = "D:/sfm_dense/street_1_0_query/dense/0/fused.ply"
# ply_target = "D:/sfm_dense/street_1_0_query/query.sparse.ply"
ply_target ="E:/University/Thesis/color_pd30_8bit_small_filtered.ply"

source = o3d.io.read_point_cloud(ply_source)
target = o3d.io.read_point_cloud(ply_target)
threshold = 0.05
voxel_size = 0.1  # means 5cm for the dataset

trans_init = np.asarray([
    [0.862, 0.011, -0.507, 110.9],
    [-0.139, 0.967, -0.215, 220.1],
    [0.487, 0.255, 0.835, -1.4],
    [0.0, 0.0, 0.0, 1.0]
])
# trans_init = np.asarray([
#     [1, 0, 0, 0],
#     [0, 1, 0, 0],
#     [0, 0, 1, 0],
#     [0, 0, 0, 1]
# ])
source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)


In [46]:
print("Initial alignment")
draw_registration_result(source, target, trans_init)
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, trans_init)
print(evaluation)
print()

Initial alignment
RegistrationResult with fitness=0.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 0
Access transformation to get result.



In [47]:
source_down = source_down.translate([100, 100, 0], relative=False)
source_down_np = np.asarray(source_down.points)
source_down = source_down.select_by_index(np.where(source_down_np[:, 2] < 0)[0])
voxel_size=5
result_fast = execute_global_registration(
    source_down,
    target_down,
    source_fpfh,
    target_fpfh,
    voxel_size
)
print("Global", result_fast)
draw_registration_result(source_down, target_down, result_fast.transformation)
evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, result_fast.transformation)
print("Global", evaluation)
print("Global Transformation",)
print(result_fast.transformation)
print()

Global RegistrationResult with fitness=1.000000e+00, inlier_rmse=1.531789e+00, and correspondence_set size of 6893
Access transformation to get result.
Global RegistrationResult with fitness=1.710425e-04, inlier_rmse=4.357783e-02, and correspondence_set size of 2
Access transformation to get result.
Global Transformation
[[-9.93376071e-01 -1.14900217e-01 -1.38614232e-03  1.51731754e+02]
 [ 1.14903958e-01 -9.93372027e-01 -3.01587450e-03  8.96897580e+01]
 [-1.03043037e-03 -3.15517080e-03  9.99994492e-01  3.39042202e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



In [48]:
threshold = 1
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source_down,
    target_down,
    threshold,
    result_fast.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
print("")
draw_registration_result(source, target, reg_p2p.transformation)

Apply point-to-point ICP
RegistrationResult with fitness=5.422893e-01, inlier_rmse=5.878168e-01, and correspondence_set size of 3738
Access transformation to get result.
Transformation is:
[[-9.92221306e-01 -1.24363189e-01 -5.53861630e-03  1.52827069e+02]
 [ 1.24163230e-01 -9.91869106e-01  2.79135902e-02  8.84513402e+01]
 [-8.96500549e-03  2.70087664e-02  9.99594996e-01  5.00113186e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



In [15]:
print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
    source,
    target,
    threshold,
    result_fast.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPlane()
)
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
print("")
draw_registration_result(source, target, reg_p2l.transformation)

Apply point-to-plane ICP
RegistrationResult with fitness=2.475915e-01, inlier_rmse=3.599910e-01, and correspondence_set size of 60986
Access transformation to get result.
Transformation is:
[[-0.67361182  0.20082428 -0.71127823  5.85798277]
 [-0.17337658 -0.97845893 -0.11206557  0.67536355]
 [-0.71846202  0.0478303   0.69391973  0.26015355]
 [ 0.          0.          0.          1.        ]]

